In [36]:
import pandas as pd
import numpy as np
import re

In [2]:
table = pd.read_csv('/home/student.unimelb.edu.au/shorte1/Documents/ACF_consulting/EPBC_notices.csv')

In [3]:
table['Approval Holder'] = table['Title of referral'].apply(lambda x: x.split('/')[0])
table['Industry'] = table['Title of referral'].apply(lambda x: x.split('/')[1])
table['Holder Address'] = table['Title of referral'].apply(lambda x: x.split('/')[2])
table['State'] = table['Title of referral'].apply(lambda x: x.split('/')[3])
table['Description'] = table['Title of referral'].apply(lambda x: x.split('/')[4])

In [4]:
import urllib
import json
url = 'https://data.gov.au/data/api/3/action/datastore_search?resource_id=eb1e6be4-5b13-4feb-b28e-388bf7c26f93&limit=200000'  
fileobj = urllib.request.urlopen(url)
ACNC_register = pd.DataFrame(json.loads(fileobj.read())['result']['records'])

In [112]:
del ACNC_register

In [126]:
url = 'https://data.gov.au/data/api/3/action/datastore_search?resource_id=cb7e4eb5-ed46-4c6c-97a0-4532f4479b7d&limit=300000'
fileobj = urllib.request.urlopen(url)
ASIC_register = pd.DataFrame(json.loads(fileobj.read())['result']['records'])

In [144]:
# ASIC_register = pd.read_csv('/home/student.unimelb.edu.au/shorte1/Documents/ACF_consulting/COMPANY_202103.csv', sep='\t', encoding = "ISO-8859-1", dtype={'Previous State of Registration': str, 'State Registration number': str, 'Modified since last report': str, 'Current Name Indicator': str, 'ABN': str, 'Current Name': str, 'Current Name Start Date': str})
ASIC_register = pd.read_csv('/home/student.unimelb.edu.au/shorte1/Documents/ACF_consulting/COMPANY_202103.csv', sep='\t', encoding = "ISO-8859-1", dtype=str)

In [148]:
ASIC_register['Company Name']

0          TGR BIOSCIENCES PTY LTD                       ...
1          DELVADO AGENCIES PTY. LTD.                    ...
2          ILLUMINATED SOLUTIONS PTY LTD                 ...
3          FINTONE AUSTRALIA PTY. LTD.                   ...
4          SINGH MARKETING CORPORATION PTY LTD           ...
5          LAREDO PTY. LTD.                              ...
6          DR CHRIS HARDWICKE PTY. LTD                   ...
7          OMEGA DEFENCE & SECURITY EQUIPMENT PTY. LIMITE...
8          ROADWAY MUFFLER CENTRE PTY LTD                ...
9          ALEXANDER GRAHAM PTY. LTD.                    ...
10         ALEXANDER & SYMONDS SPATIAL PTY LTD           ...
11         FELLOWS FISH PTY. LTD.                        ...
12         MUSCOCOMBE PTY. LTD.                          ...
13         HILMAK ENTERPRISES PTY. LTD.                  ...
14         MIRACLELAWN PTY. LTD.                         ...
15         CAPTAIN STEVE PELECANOS & ASSOCIATES PTY LTD  ...
16         MIENREACH PTY

In [7]:
ACNC_register['Charity_Legal_Name'].values[10]

array(['Sydney Missionary & Bible College', 'Integricare',
       'Australian Missionary Tidings', ...,
       'Pmwu Of The Presbyterian Womens Assoc NSW',
       'Ashfield Chinese Baptist Church',
       'Kalgoorlie Baptist Church Incorporated'], dtype=object)

In [65]:
res = [x for x in table['Approval Holder'].values if re.search('(?i)department', x)]

In [66]:
res

['Department of Transport',
 'Department of Transport',
 'Department of Transport',
 'Department of State Growth',
 'Department of State Growth',
 'Department of Transport',
 'Department for Infrastructure and Transport, Government of South Australia',
 'Department of Transport and Main Roads',
 'Department of Transport and Main Roads',
 'Western Australian Department of Education',
 'Department of Transport',
 'Department of Local Government, Sport and Cultural Industries',
 'Department for Infrastructure and Transport, Government of South Australia',
 'DEPARTMENT OF JUSTICE AND REGULATION',
 'Department of Transport',
 'Department of Transport',
 'Department of Transport and Main Roads',
 'Department of Local Government, Sport and Cultural Industries',
 'Department of Transport',
 'Department of Transport and Main Roads',
 'Queensland Department of Transport and Main Roads',
 'Department of State Growth',
 'Department of Transport and Main Roads',
 'Department of Transport and Main R

In [72]:
table['Approval Holder']

0                             AGI Operations Pty Limited
1                           NARRE WARREN CENTRAL PTY LTD
2                     Co-Operative Bulk Handling Limited
3           AUSTRALIA PACIFIC AIRPORTS MELBOURNE PTY LTD
4                                  BURNETT WATER PTY LTD
5           APA Northern Goldfields Interconnect Pty Ltd
6                                    Snowy Hydro Limited
7                          AGL SA Generation Pty Limited
8                                    LA TROBE UNIVERSITY
9                                  Sylvanvale Foundation
10       Gladstone Area Water Board and SunWater Limited
11     Western Australian Land Authority trading as D...
12                              S,C & B Catalano Pty Ltd
13                             Eurobodalla Shire Council
14                               Department of Transport
15                               Department of Transport
16                              Pioneer North Queensland
17                             

In [6]:
from fuzzywuzzy import fuzz

In [67]:
# test_NFP = table['Approval Holder'].iloc[175]
test_NFP = 'DEPARTMENT OF JUSTICE AND REGULATION'

In [78]:
sector = ['Not yet categorised.']*len(table['Approval Holder'].values)

for i in range(len(table['Approval Holder'].values[1:10])):
    ratio_list = np.zeros(len(ACNC_register['Charity_Legal_Name'].values))
    for j in range(len(ACNC_register['Charity_Legal_Name'].values)):
        ratio = fuzz.token_sort_ratio(
            table['Approval Holder'].values[i], 
            ACNC_register['Charity_Legal_Name'].values[j]
        )
        if ratio > 98:
            sector[i] = 'Not for Profit'
            break
        else: 
            ratio_list[i] = ratio
            
    if np.amax(ratio_list) > 95:
        sector[i] = 'Not for Profit (Confirm Manually)'

In [81]:
table['Approval Holder'][[2,8]]

2    Co-Operative Bulk Handling Limited
8                   LA TROBE UNIVERSITY
Name: Approval Holder, dtype: object

In [71]:
# np.argmax(ratio_list)
ACNC_register['Charity_Legal_Name'][48893]

'Evolution and development'

In [31]:
ACNC_register['Charity_Legal_Name'].values[14]

'Australian Nurses Memorial Centre'

In [18]:
ACNC_register['Charity_Legal_Name'].values[22884]

'Catholic Parish Of Mary Help Of Christians Sawtell'

In [11]:
Ratio = fuzz.ratio(test_NFP_1.lower(),test_NFP_2.lower())
Partial_Ratio = fuzz.partial_ratio(test_NFP_1.lower(),test_NFP_2.lower())
Token_Ratio = fuzz.token_sort_ratio(test_NFP_1,test_NFP_2)

In [12]:
print(Ratio)
print(Partial_Ratio)
print(Token_Ratio)

44
45
44
